In [6]:
###TKAN original

import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tkan import TKAN
import os
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=0'
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

print("TensorFlow version:", tf.__version__)
print("Keras version:", tf.keras.__version__)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Verify GPU availability (should be 0 now)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Data import
site_number = 3
data = pd.read_excel(f'Wind farm site {site_number} (Nominal capacity-99MW).xlsx')

# Convert time column to datetime and correct invalid times
data['Time(year-month-day h:m:s)'] = data['Time(year-month-day h:m:s)'].apply(lambda x: str(x).replace(' 24:', ' 00:'))
data['Time(year-month-day h:m:s)'] = pd.to_datetime(data['Time(year-month-day h:m:s)'], format='%Y-%m-%d %H:%M:%S')

# Set time column as index
data.set_index('Time(year-month-day h:m:s)', inplace=True)

# Strip leading/trailing spaces from column names
data.columns = data.columns.str.strip()

# Handle NaN values
data.fillna(method='ffill', inplace=True)

# Normalize the features
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns, index=data.index)

def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :-1])  # All features except the last (target) column
        y.append(data[i, -1])  # Target column (Power output)
    return np.array(X), np.array(y)

# Number of timesteps in the input sequence
n_steps = 24  # For example, 24*15min = 6 hours of historical data to predict the next value

# Prepare input/output sequences
X, y = create_sequences(data_scaled.values, n_steps)

# Splitting data into training and testing sets (80-20 split)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# # Building the simple TKAN model
# def simple_TKAN_model(input_shape):
#     model = Sequential()
#     model.add(TKAN(units=50, tkan_activations=None, return_sequences=True, input_shape=input_shape))
#     model.add(Dropout(0.75))
#     model.add(TKAN(units=50, tkan_activations=None))
#     model.add(Dropout(0.75))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mse', jit_compile=False)
#     return model


# from keras.models import Model
# from keras.layers import Input, Dropout, Dense

def simple_TKAN_model(input_shape):
    inputs = Input(shape=input_shape)
    x = TKAN(units=50, tkan_activations=None, return_sequences=True)(inputs)
    x = Dropout(0.75)(x)
    x = TKAN(units=50, tkan_activations=None)(x)
    x = Dropout(0.75)(x)
    outputs = Dense(1)(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', jit_compile=False)
    return model


# Instantiate and compile the model
model_SIMPLE_TKAN = simple_TKAN_model((n_steps, X_train.shape[2]))
model_SIMPLE_TKAN.summary()

history = model_SIMPLE_TKAN.fit(X_train, y_train, epochs=25, batch_size=100, validation_split=0.2, verbose=1)

# Ensure the directory exists
os.makedirs('wind_results', exist_ok=True)

# Save validation loss
np.save(f'wind_results/validation_loss_SIMPLE_TKAN_{site_number}_{n_steps}.npy', history.history['val_loss'])

# Plotting training and validation loss
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Test')
plt.title('Training vs Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(f'wind_results/SIMPLE_TKAN_{site_number}_{n_steps}.png', format='png')
plt.show()

# Predictions
predictions_1 = model_SIMPLE_TKAN.predict(X_test)
np.save(f'wind_results/prediction_SIMPLE_TKAN_{site_number}_{n_steps}.npy', predictions_1)  # Save predictions

rmse = np.sqrt(mean_squared_error(y_test, predictions_1))
mae = mean_absolute_error(y_test, predictions_1)
epsilon = 1e-8  # Small constant
mape = np.mean(np.abs((y_test - predictions_1) / (y_test + epsilon))) * 100
r2 = r2_score(y_test, predictions_1)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R2 Score: {r2}")

# Save results to file
results_file = f"wind_results/SIMPLE_TKAN_{n_steps}.txt"
with open(results_file, "a") as file:
    file.write(f"Site {site_number}:\n")
    file.write(f"RMSE: {rmse}\n")
    file.write(f"MAE: {mae}\n")
    file.write(f"R2 Score: {r2}\n")
    file.write("\n")


TensorFlow version: 2.16.1
Keras version: 3.4.1.dev2024072903
Num GPUs Available:  0


C:\Users\ardahal\AppData\Local\Temp\ipykernel_32576\2234561268.py:51: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
